In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [11]:
pages = []  # Список для хранения всех страниц с квартирами
for i in range(1, 22):  # Оказалось, что страниц 21 штучка
    flats_page = BeautifulSoup(requests.get(
        'http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?p=' + str(i)
    ).text, 'lxml')  # Нагло забираем страничку
    pages.append(flats_page)  # И добавляем её в списочек

In [14]:
all_links_list = []  # Список для ссылок на конкретные квартиры
all_ids_list = []  # Список для id квартир
for page in pages:  # Проходимся по страничкам
    print(page)
    table = page.find('div', {'class': 'wrapper'}).find('table', {'id':'tbl_10043'})  # Читаем таблицу с квартирами с нашей страницы
    print(table)
    for tr in table.find_all('tr'):  # Проходимся по строчкам таблицы
      if len(tr) != 13:  # Находим строчки с нужной информацией
          continue
      id = int(tr.find_all('td', recursive=False)[0].input['value'])  # Считываем id из input
      link = tr.find_all('td', recursive=False)[4].a['href']  # Считываем ссылочку из address
      all_links_list.append(link)  # Заносим всё, что нашли в наши списочки
      all_ids_list.append(id)

Выходные данные были обрезаны до нескольких последних строк (5000).
</tr>
<tr>
<td class="notebook-column"><input name="cs_note_tg" type="checkbox" value="18947572"/></td>
<td class="date">10 дек<br/><span class="time">07:35</span></td><td class="image"><div class="image-block"><span class="ibf" id="ibfbut_18947572" title="Добавить в избранное"></span><a href="/realty/prodazha-kvartir/prodam-kvartiru-18947572.html"><img alt="" src="http://s10.citystar.ru/0u/9c0d9e68584ba8ff919cdd629e6ff726-crop-87x65.jpg"/></a></div></td><td class="flat-type"><table style="border:none;"><tr><td style="border:none;"><b></b></td><td style="border:none;"></td></tr></table></td><td class="address"><a href="/realty/prodazha-kvartir/prodam-kvartiru-18947572.html">Уральская, 44</a></td><td class="floor">1<span class="slash">/</span>1</td><td class="square">124,3<span class="slash">/</span>66<span class="slash">/</span>13,5 <span class="colored-text-1">м2</span></td><td class="price"><span>4 099 000</span><spa

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
columns = ['Ссылка', 'Цена', 'Район', 'Адрес', 'Этаж',
           'Высота потолка', 'Планировка', 'Площадь',
           'Состояние квартиры', 'Кол-во комнат', 'Описание']
data = pd.DataFrame(index=all_ids_list, columns=columns)  # Создаем DataFrame
data['Ссылка'] = all_links_list  # Сразу заполняем столбик ссылок
data

In [ ]:
for i in range(len(all_ids_list)):  # Проходимся по всем страницам с конкретными кавртирами
  flat_page = BeautifulSoup(requests.get(
        'http://magnitogorsk-citystar.ru' + all_links_list[i]
    ).text, 'lxml')
  div = flat_page.find('div', 'adv-main-data')  # Находим контейнер с необходимой инофрмацией
  key_list = [key.get_text() for key in div.find_all('td', 'field-title')]  # Список с ключами для словаря с информацией о квартире (заголовки)
  value_list = [value.get_text().replace('\xa0', '') for value in div.find_all('td', 'field')]  # Список уже с самой информацией (цена, площадь и т.д.)
  data_dict = dict(zip(key_list, value_list))  # Склеиваем
  if div.find('td', 'note'):  # Отдельно проверяем наличия описания квартиры
        data_dict['Описание'] = div.find('td', 'note').get_text()  # И добваляем, если нашли
  data.loc[all_ids_list[i], 'Цена':] = data_dict  # Обновляем строку с квартирой в нашей таблице

In [ ]:
data.head()

In [ ]:
data.to_csv('magnitogorsk_parsing.csv')
data.to_excel('magnitogorsk_parsing.xlsx')
files.download('magnitogorsk_parsing.csv')
files.download('magnitogorsk_parsing.xlsx')